<a href="https://colab.research.google.com/github/RifatMuhtasim/Data_Science_Workflow/blob/main/3.2.Transform_Skewed_Columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Identify Skewed Columns

In [ ]:
import scipy.stats as stat
from scipy.stats import skew, kurtosis
import statsmodels.api as sm
import pylab


def QQPlot(df, x):
    sm.qqplot(df[x], line='45')
    print("Skew:", skew(df[x]))
    print("Kurtosis:", kurtosis(df[x]))
    plt.show()


def Diagnostic_plot(df, feature):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    sns.histplot(x=df[feature],  kde=True)
    plt.subplot(1, 2, 2)
    stat.probplot(df[feature], dist="norm", plot = pylab)

    print("Skew:", skew(df[feature]))
    print("Kurtosis:", kurtosis(df[feature]))
    plt.title(feature)
    plt.show()

In [ ]:
def Identify_skewed_columns(df):
    df_copy = df.copy()

    def skewed_columns(df, threshold=0.5):
        skewed_columns = []
        for col in df.columns:
            if df[col].dtype in ['float64', 'int64']:
                skewness = df[col].skew()
                if abs(skewness) > threshold:
                    skewed_columns.append((col, skewness))
        return skewed_columns


    skewed_cols = skewed_columns(df)
    print("Skewed Columns:")
    for col, skewness in skewed_cols:
        print(f"{col} Skewness = {skewness}")

    skewness_list = []
    for col, skewness in skewed_cols:
        skewness_list.append(col)
    return skewness_list


skew_columns = Identify_skewed_columns(df= my_df)

In [ ]:
# Show the skewed Columns

for x in skew_columns:
    print("Columns: ", x)
    Diagnostic_plot(df= my_df, feature=x)

# 2. Handle Skewed Columns

In [ ]:
from scipy.stats import skew, boxcox


def Find_lowest_skewness(df, columns):
    lowest_skewness = {}
    for x in columns:
        log = abs(skew(np.log(df[x])))
        reciprocal = abs(skew(1/(df[x])))
        sqrt = abs(skew((df[x])**(1/2)))
        exp = abs(skew(np.log(df[x])**(1/1.2)))
        # boxcox_trans, _ = boxcox(df[x]+1)
        # boxcox_skew = abs(skew(boxcox_trans))

        # Find the minimum skewness value and the corresponding transformation
        # min_skew = min(log, reciprocal, sqrt, exp, boxcox_skew)
        min_skew = min(log, reciprocal, sqrt, exp)

        if min_skew == log:
            transform = 'log'
        elif min_skew == reciprocal:
            transform = 'reciprocal'
        elif min_skew == sqrt:
            transform = 'sqrt'
        elif min_skew == exp:
            transform = 'exponential'
        elif min_skew == boxcox:
            transform = 'boxcox'
        else:
            transform = "delete"

        lowest_skewness[x] = (min_skew, transform)

    return lowest_skewness

In [ ]:
result = Find_lowest_skewness(df = my_df, columns= skew_columns)
for col, (skewness, transform) in result.items():
    print(f"{col} :- Lowest Skewness = {skewness} (Transform: {transform})")

## Transform skewed columns

In [ ]:
# logarithmic transformation
my_df['regression_cube_log'] = np.log(my_df['regression_cube'])

# reciprocal transformation
my_df['regression_cube_reciprocal'] = 1/(my_df['regression_cube'])

# square root transformation
my_df['regression_cube_sqrt'] = (my_df['regression_cube'])**(1/2)

# exponential transformation (more general, you can use any exponent)
my_df['regression_cube_exp'] = my_df['regression_cube']**(1/1.2)

# boxcox transformation
import scipy.stats as stat
my_df['regression_cube_boxcox'], _ = stat.boxcox(my_df['regression_cube']+1)

In [ ]:
# Remove columns

remove_columns_list = ['B', 'C']
df = my_df.drop(remove_columns_list, axis=1)